In [1]:
import hidet

def matmul_func(m_size, n_size, k_size):
    from hidet.lang import attr, f32
    from hidet.lang.cuda import threadIdx, blockIdx, blockDim
    from hidet.transforms.tools import add_packed_func

    def ceil_div(a, b):
        return (a + b - 1) // b

    tile_size = 2

    with hidet.script_module() as script_module:

        @hidet.script
        def kernel(
                a: f32[m_size, k_size],
                b: f32[k_size, n_size],
                c: f32[m_size, n_size]
        ):
            attr.func_kind = 'cuda_kernel'
            attr.cuda_block_dim = (tile_size, tile_size)
            attr.cuda_grid_dim = ceil_div(m_size, tile_size), ceil_div(n_size, tile_size)
            i = threadIdx.x + blockIdx.x * blockDim.x
            j = threadIdx.y + blockIdx.y * blockDim.y
            if i < m_size and j < n_size:
                acc = f32(0.0)
                for k in range(k_size):
                    acc += a[i, k] * b[k, j]
                c[i, j] = acc

    ir_module = script_module.ir_module()
    add_packed_func(ir_module, func=kernel, pack_func_name='matmul')
    return hidet.driver.build_ir_module(ir_module, func_name='matmul')

m_size, n_size, k_size = 5, 5, 5
matmul = matmul_func(m_size, n_size, k_size)
a = hidet.randn([m_size, k_size]).cuda()
b = hidet.randn([k_size, n_size]).cuda()
c = hidet.empty([m_size, n_size]).cuda()

latency = hidet.utils.benchmark_func(lambda: matmul(a, b, c))
print('Latency: {:.2f} ms'.format(latency))

Latency: 0.01 ms


In [3]:
import numpy as np
import numpy.testing

np_a = a.cpu().numpy()
np_b = b.cpu().numpy()
np_c = np.matmul(np_a, np_b)
numpy.testing.assert_allclose(c.cpu().numpy(), np_c, rtol=1e-5, atol=1e-5)